# Ranking school examination results using multivariate hierarchical models

In [1]:
import numpy as np
import scipy.stats as sp
import matplotlib.pyplot as plt

In [2]:
import json 

with open("data/data.json") as file:
    data = json.load(file)
    
with open("data/init.json") as file:
    init = json.load(file)
    
with open("data/params.json") as file:
    params = json.load(file)

In [3]:
for idx, item in data.items():
    data[idx] = np.array(item).reshape(-1)

for idx, item in init.items():
    init[idx] = np.array(item).reshape(-1)
    
for idx, item in params.items():
    params[idx] = np.array(item).reshape(-1)

In [4]:
data.keys(), init.keys(), params.keys()

(dict_keys(['Gender', 'LRT', 'M', 'N', 'R', 'school', 'School_denom', 'School_gender', 'Y', 'VR']),
 dict_keys(['theta', 'phi', 'gamma', 'beta', 'Omega', 'alpha']),
 dict_keys(['R_shape', 'School_denom_shape', 'School_gender_shape', 'Venom_shape', 'Omega_shape', 'alpha_shape']))

In [20]:
hyperparams = dict()
hyperparams["tau0"] = 1e-3
hyperparams["sd_proposal"] = 1e-3

In [21]:
def sampler(n, data, init, params, hyperparams):
    
    # initialisation de la chaine 
    chain = dict()
    for key, param in init.items():
        chain[key] = np.zeros((n+1, param.shape[0]))
        chain[key][0, :] = param
    
    # définition des constantes
    Y = data["Y"]
    
    N   = Y.shape[0]
    n_beta = init["beta"].shape[0]
    n_schools = np.max(data["school"])
    
    LRT_sum = np.sum(data["LRT"])
    tau0 = hyperparams["tau0"]
    sd_proposal = hyperparams["sd_proposal"]
    
    T0 = np.linalg.inv(init["Omega"].reshape(3, 3))
    R_inv = np.linalg.inv(data["R"].reshape(3, 3))
    
    # définition de v
    LRT = data["LRT"].reshape(-1, 1)
    VR = data["VR"].reshape(N, -1, order="F")
    Girl = data["Gender"].reshape(-1, 1)
    School_gender = data["School_gender"].reshape(N, -1, order="F")
    School_denom = data["School_denom"].reshape(N, -1, order="F")

    v = np.concatenate((LRT**2, VR[:, [1]], Girl , School_gender, School_denom), axis=1)
    
    # définition de u
    u = np.concatenate((np.ones((N, 1)), LRT, VR[:, [1]]), axis=1)
    
    # initialisation de mu et tau
    mu_current  = np.zeros(N) 
    mu1_current  = np.zeros(N) 
    mu2_current  = np.zeros(N) 
    
    # mise à jour des paramètres
    for i in range(1, n+1):
        print(i, end=" ")
        # mise à jour de alpha: normale
        ## calcul de tau
        theta_current = chain["theta"][i-1, :]
        phi_current = chain["phi"][i-1, :]
        tau_current = np.exp(theta_current + phi_current * LRT).flatten()
        
        ## variables intervenant dans la loi de alpha
        beta_current = chain["beta"][i-1, :]
        mu2_current = np.dot(v, beta_current) 
        
        gamma_current = chain["gamma"][i-1, :]
        Omega_current = chain["Omega"][i-1, :].reshape(3, 3)
        T_current = np.linalg.inv(Omega_current)
        for j in range(1, n_schools+1):
            school_j = (data["school"] == j)
            tau_j = tau_current[school_j]
            u_j = u[school_j, :]
            v_j = v[school_j, :]
            Y_j = Y[school_j]
            
            n_j = len(Y_j)
            alphaj_cov_inv = T_current
            for k in range(n_j):
                u_kj =  u[[k], :]
                alphaj_cov_inv = alphaj_cov_inv + tau_j[k] * np.dot(u_kj.T, u_kj)
            alphaj_cov = np.linalg.inv(alphaj_cov_inv)
            
            alphaj_mean = np.dot(T_current, gamma_current)
            for k in range(n_j):
                alphaj_mean = alphaj_mean + tau_j[k] * (Y_j[k] - np.dot(beta_current, v_j[k, :])) * u[k, :]
            alphaj_mean = np.dot(alphaj_cov, alphaj_mean)
            
            alphaj_proposal = np.random.multivariate_normal(alphaj_mean, alphaj_cov)
            chain["alpha"][i,(3*(j-1)):(3*j)] = alphaj_proposal
        
        alpha_current = chain["alpha"][i, :]
        
        # mise à jour de mu_1 (car elle dépend de alpha)
        for j in range(1, n_schools + 1):
            school_j = (data["school"] == j)
            alpha_j = alpha_current[(3*(j-1)):(3*j)]
            mu1_current[school_j] = np.dot(u[school_j, :], alpha_j) 
        mu_current = mu1_current + mu2_current
        
        # mise à jour de beta: normale multivariée        
        beta_cov_inv = tau0 * np.eye(n_beta) 
        for k in range(N):
            beta_cov_inv = beta_cov_inv + tau_current[k] * np.dot(v[[k], :].T, v[[k], :])
        beta_cov = np.linalg.inv(beta_cov_inv)
        
        tmp = np.zeros(v.shape)
        for k in range(N):
            tmp_k = tau_current[k] * (Y[k] - np.dot(alpha_j, u[k, :]))
            tmp_k = tmp_k * v[k, :]
            tmp[k, :] = tmp_k
        beta_mean = np.sum(tmp, axis=0)
        beta_mean = np.dot(beta_cov, beta_mean)
        
        beta_proposal = np.random.multivariate_normal(beta_mean, beta_cov)
        
        chain["beta"][i, :] = beta_proposal
        beta_current = beta_proposal
        mu2_current = np.dot(v, beta_current)
        mu_current = mu1_current + mu2_current
        
        # mise à jour de phi: mcmc 
        # phi peut prendre toutes les valeurs réelles, 
        # donc on utilise le "random walk M.-H. scheme"
        phi_proposal = phi_current + np.random.normal(scale=sd_proposal)
        
        phi_top = - tau0 * np.power(phi_proposal, 2)
        phi_top += LRT_sum * phi_proposal
        phi_top -= np.exp(theta_current) * np.sum(
            np.exp(phi_proposal*data["LRT"]) * np.power(data["Y"]-mu_current, 2)
        )
        phi_top *= .5
        
        phi_bottom = -tau0 * np.power(phi_current, 2)
        phi_bottom += LRT_sum * phi_current
        phi_bottom -= np.exp(theta_current) * np.sum(
            np.exp(phi_current*data["LRT"]) * np.power(data["Y"]-mu_current, 2)
        )
        phi_bottom *= .5
        
        phi_acceptance = np.exp(phi_top-phi_bottom)
        if phi_acceptance < np.random.rand():
            chain["phi"][i,:] = phi_proposal
            phi_current = phi_proposal
            tau_current = np.exp(theta_current + phi_current * LRT).flatten()
        else:
            chain["phi"][i,:] = phi_current

        # mise à jour de theta: mcmc 
        # theta peut prendre toutes les valeurs réelles, 
        # donc on utilise le "random walk M.-H. scheme"        
        theta_proposal = theta_current + np.random.normal(scale=sd_proposal)
        
        theta_top = (-tau0/2) * np.power(theta_proposal, 2)
        theta_top += (N/2) * theta_proposal
        theta_top -= (np.exp(theta_proposal)/2) * np.sum(
            np.exp(phi_current*data["LRT"]) * np.power(data["Y"]-mu_current, 2)
        )
        
        theta_bottom = (-tau0/2) * np.power(theta_current, 2)
        theta_bottom += (LRT_sum/2) * theta_current
        theta_bottom -= (np.exp(theta_current)/2) * np.sum(
            np.exp(phi_current*data["LRT"]) * np.power(data["Y"]-mu_current, 2)
        ) 
        
        theta_acceptance = np.exp(theta_top-theta_bottom)
        if theta_acceptance < np.random.rand():
            chain["theta"][i,:] = theta_proposal
            theta_current = theta_proposal
            tau_current = np.exp(theta_current + phi_current * LRT).flatten()
        else:
            chain["theta"][i,:] = theta_current
        
        # mise à jour de gamma: normale
        alpha_current_sum = np.zeros(3)
        for j in range(1, n_schools+1):
            alpha_current_sum = alpha_current_sum + alpha_current[(3*(j-1)):(3*j)]
        
        gamma_cov = np.linalg.inv(T0 + n_schools*T_current)
        gamma_mean = np.dot(gamma_cov, np.dot(T_current, alpha_current_sum))
        
        gamma_proposal = np.random.multivariate_normal(gamma_mean, gamma_cov)
        
        chain["gamma"][i, :] = gamma_proposal
        gamma_current = gamma_proposal
        
        # mise à jour de T=inverse de Omega: mcmc       
        for (m, n) in ((0,0), (0,1), (0,2), (1,1), (1,2), (2,2)):
            # boucle pour garantir que T reste définie positive
            while True:
                Tmn =  np.random.normal(T_current[m, n], scale=sd_proposal)

                T_proposal = np.copy(T_current)
                T_proposal[m, n] = Tmn
                T_proposal[n, m] = Tmn
                
                if np.all(np.linalg.eigvals(T_proposal) > 0):
                    Omega_proposal = np.linalg.inv(T_proposal)
                    break
            
            T_top = (n_schools-1) * np.log(np.linalg.det(T_proposal))
            T_top -= np.sum(np.diag(np.dot(R_inv, T_proposal))) 
            
            T_bottom = (n_schools-1) * np.log(np.linalg.det(T_current))
            T_bottom -= np.sum(np.diag(np.dot(R_inv, T_current))) 
            for j in range(1, n_schools+1):
                alpha_j = alpha_current[(3*(j-1)):(3*j)]
                diff_j = (gamma_current - alpha_j).reshape(-1, 1)
                T_top -= np.dot(diff_j.T, np.dot(T_proposal, diff_j))
                T_bottom -= np.dot(diff_j.T, np.dot(T_current, diff_j))
            T_top *= .5
            T_bottom *= .5
            
            T_acceptance = np.exp(T_top - T_bottom)
            
            if T_acceptance < np.random.rand():
                chain["Omega"][i, :] = Omega_proposal.flatten()
                T_current = T_proposal
                Omega_current = Omega_proposal
            else:
                chain["Omega"][i, :] = Omega_current.flatten()
        
    return chain

In [22]:
 chain = sampler(1000, data, init, params, hyperparams)

1 2 3 4 5 6 7 8 9 

/tmp/ipykernel_4863/697893419.py:127: RuntimeWarning: overflow encountered in exp
  phi_acceptance = np.exp(phi_top-phi_bottom)


10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 

/tmp/ipykernel_4863/697893419.py:152: RuntimeWarning: overflow encountered in exp
  theta_acceptance = np.exp(theta_top-theta_bottom)


39 40 41 42 43 44 45 46 47 48 49 50 51 52 

...

53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 306 307 308 309 310 311 312 313 314

/home/meh/anaconda3/envs/minimal/lib/python3.10/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/tmp/ipykernel_4863/697893419.py:127: RuntimeWarning: invalid value encountered in subtract
  phi_acceptance = np.exp(phi_top-phi_bottom)
/tmp/ipykernel_4863/697893419.py:152: RuntimeWarning: invalid value encountered in subtract
  theta_acceptance = np.exp(theta_top-theta_bottom)
/tmp/ipykernel_4863/697893419.py:116: RuntimeWarning: overflow encountered in multiply
  np.exp(phi_proposal*data["LRT"]) * np.power(data["Y"]-mu_current, 2)
/tmp/ipykernel_4863/697893419.py:123: RuntimeWarning: overflow encountered in multiply
  np.exp(phi_current*data["LRT"]) * np.power(data["Y"]-mu_current, 2)
/tmp/ipykernel_4863/697893419.py:143: RuntimeWarning: overflow encountered in multiply
  np.exp(phi_current*data["LRT"]) * np.power(data["Y"]-mu_current, 2)
/tmp/ipykernel_4863/697893419.py:149: RuntimeWar

610 611 612 613 614 615 616 617 

/tmp/ipykernel_4863/697893419.py:116: RuntimeWarning: overflow encountered in power
  np.exp(phi_proposal*data["LRT"]) * np.power(data["Y"]-mu_current, 2)
/tmp/ipykernel_4863/697893419.py:123: RuntimeWarning: overflow encountered in power
  np.exp(phi_current*data["LRT"]) * np.power(data["Y"]-mu_current, 2)
/tmp/ipykernel_4863/697893419.py:143: RuntimeWarning: overflow encountered in power
  np.exp(phi_current*data["LRT"]) * np.power(data["Y"]-mu_current, 2)
/tmp/ipykernel_4863/697893419.py:149: RuntimeWarning: overflow encountered in power
  np.exp(phi_current*data["LRT"]) * np.power(data["Y"]-mu_current, 2)
/tmp/ipykernel_4863/697893419.py:195: RuntimeWarning: overflow encountered in subtract
  T_top -= np.dot(diff_j.T, np.dot(T_proposal, diff_j))
/tmp/ipykernel_4863/697893419.py:196: RuntimeWarning: overflow encountered in subtract
  T_bottom -= np.dot(diff_j.T, np.dot(T_current, diff_j))
/tmp/ipykernel_4863/697893419.py:200: RuntimeWarning: invalid value encountered in subtract
  T

618 619 620 621 622 623 624 625 626 627 628 629 630 631 632 633 634 635 636 637 638 639 640 641 642 643 644 645 646 647 648 649 650 651 652 653 654 655 656 657 658 659 660 661 662 663 664 665 666 667 668 669 670 671 672 673 674 675 676 677 678 679 680 681 682 683 684 685 686 687 688 689 690 691 692 693 694 695 696 697 698 699 700 701 702 703 704 705 706 707 708 709 710 711 712 713 714 715 716 717 718 719 720 721 722 723 724 725 726 727 728 729 730 731 732 733 734 735 736 737 738 739 740 741 742 743 744 745 746 747 748 749 750 751 752 753 754 755 756 757 758 759 760 761 762 763 764 765 766 767 768 769 770 771 772 773 774 775 776 777 778 779 780 781 782 783 784 785 786 787 788 789 790 791 792 793 794 795 796 797 798 799 800 801 802 803 804 805 806 807 808 809 810 811 812 813 814 815 816 817 818 819 820 821 822 823 824 825 826 827 828 829 830 831 832 833 834 835 836 837 838 839 840 841 842 843 844 845 846 847 848 849 850 851 852 853 854 855 856 857 858 859 860 861 862 863 864 865 866 867 

In [25]:
chain["phi"]

array([[0.01      ],
       [0.01      ],
       [0.01007099],
       ...,
       [0.25407415],
       [0.25407415],
       [0.25407415]])